In [18]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model, metrics, preprocessing
import statsmodels.api as sm
from statsmodels.regression.linear_model import RegressionResults
import geopy.distance

In [60]:
data_dir = os.getcwd() + '/data'

modeling_df = pd.read_csv("{}/cleaned_modeling_data.csv".format(data_dir))

pred_df = modeling_df.drop(["num_testtakers", "num_offered", "pct_8th_graders_offered", "perc_testtakers", "perc_testtakers_quartile"], axis=1)
response_df = modeling_df["num_testtakers"].fillna(0)

pred_train, pred_test, response_train, response_test = train_test_split(pred_df, response_df, test_size=0.2, random_state=223)

In [67]:
# col_name = "Economic Need Index"
# print(modeling_df[col_name].describe())
# print(modeling_df[col_name].notnull().sum())
# print(modeling_df[col_name].isnull().sum())

# x_train = pred_train[col_name].to_frame().values
# x_test = pred_test[col_name].to_frame().values

# plt.xlabel(col_name)
# plt.ylabel("# Testtakers - 2017")
# plt.scatter(x_train, response_train, alpha = 0.5)
# plt.show()

# pred_regr = linear_model.LinearRegression()
# pred_model = pred_regr.fit(x_train, response_train)

# y_pred = regr.predict(x_test)
# # The coefficients
# print('Coefficients: \n', pred_regr.coef_)
# # The mean squared error
# print("Mean squared error: %.2f"
#       % mean_squared_error(response_test, y_pred))
# # Explained variance score: 1 is perfect prediction
# print('Variance score: %.2f' % r2_score(response_test, y_pred))
print(type(pred_train))
example_cols = pred_train[["Average ELA Proficiency","Average Math Proficiency"]]
#preds_to_stdize_train = pred_train[cols_to_standardize].dropna()
#print(preds_to_stdize_train.isnull().sum())
print(example_cols.mean())
print(example_cols.std())
#example_col_scaled = preprocessing.scale(example_col)
scaler = preprocessing.StandardScaler().fit(pred_train[["Average ELA Proficiency","Average Math Proficiency"]])
example_col_scaled = scaler.transform(pred_train[["Average ELA Proficiency","Average Math Proficiency"]])

test_scaled = scaler.transform(pred_test[["Average ELA Proficiency","Average Math Proficiency"]])
#print(example_col_scaled.std())
#print(example_col_scaled.mean())
#print(preds_to_stdize_train.mean())
print(pred_test[["Average ELA Proficiency","Average Math Proficiency"]][0:10])
print(test_scaled[0:10])
#scaler = preprocessing.StandardScaler().fit([example_col])
#print(scaler.mean_ )

<class 'pandas.core.frame.DataFrame'>
Average ELA Proficiency     2.531893
Average Math Proficiency    2.598174
dtype: float64
Average ELA Proficiency     0.379116
Average Math Proficiency    0.478040
dtype: float64
     Average ELA Proficiency  Average Math Proficiency
261                     2.21                      2.14
215                     2.27                      2.38
160                     2.48                      2.47
110                     2.14                      2.01
370                     2.24                      2.07
303                     2.22                      2.14
315                     2.09                      1.98
473                     2.36                      2.65
220                     2.57                      2.62
296                     2.60                      3.13
[[-0.85000986 -0.95951067]
 [-0.69157033 -0.45690096]
 [-0.13703196 -0.26842232]
 [-1.03485598 -1.23175759]
 [-0.7707901  -1.10610517]
 [-0.82360327 -0.95951067]
 [-1.16688893 -1.

In [35]:
#sm_x_train = pred_train[["borough_bronx", "borough_manhattan", "borough_queens", "borough_staten_island"]]
sm_x_train = example_col_scaled
sm_x_train = sm.add_constant(sm_x_train)

sm_model = sm.OLS(response_train, sm_x_train, missing='drop')
sm_results = sm_model.fit()
sm_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         num_testtakers   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     140.7
Date:                Thu, 02 Aug 2018   Prob (F-statistic):           2.10e-28
Time:                        05:49:15   Log-Likelihood:                -2427.2
No. Observations:                 449   AIC:                             4858.
Df Residuals:                     447   BIC:                             4867.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         43.3719      2.548     17.019      0.000      38.363      48.380
x1            30.2281      2.548     11.861      0.000      25.220      35.237
==============================================================================
Omnibus:                      265.085   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1994.761
Skew:                           2.522   Prob(JB):                         0.00
Kurtosis:                      12.010   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [93]:
invalid_preds = ["school_name", "dbn", "Address (Full)", "City", "Grades", "Grade Low", "Grade High", "SED Code", "Latitude", "Longitude", "Zip"]

numeric_pred_df = pd.DataFrame()
cat_pred_df = pd.DataFrame()
num_pred_df = pd.DataFrame()
 
for col in pred_train.columns.values:
    if col in invalid_preds:
        continue
    elif pred_train[col].dtype == "object":
        pred_rows = get_categorical_model(col, pred_train, response_train)
        cat_pred_df = cat_pred_df.append(pred_rows, ignore_index=True)
    elif pred_train[col].dtype == "float":
        print(col)
        pred_rows = get_numerical_model(col, pred_train, response_train)
        num_pred_df = num_pred_df.append(pred_rows, ignore_index=True)


pct_black_hispanic
SED Code
District
Latitude
Longitude
Zip
                            OLS Regression Results                            
Dep. Variable:         num_testtakers   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     12.23
Date:                Wed, 01 Aug 2018   Prob (F-statistic):           0.000518
Time:                        07:32:33   Log-Likelihood:                -2482.5
No. Observations:                 449   AIC:                             4969.
Df Residuals:                     447   BIC:                             4977.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

/Users/greg.bolla/.local/share/virtualenvs/kaggle-passnyc-hqRn-hsS/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/greg.bolla/.local/share/virtualenvs/kaggle-passnyc-hqRn-hsS/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/greg.bolla/.local/share/virtualenvs/kaggle-passnyc-hqRn-hsS/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/greg.bolla/.local/share/virtualenvs/kaggle-passnyc-hqRn-hsS/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
/Users/greg.bolla/.local/share/virtualenvs/kaggle-passnyc-hqRn-hsS/lib/python3.6/site-packages/statsmodels/

Student Attendance Rate
Percent of Students Chronically Absent
Rigorous Instruction %
Collaborative Teachers %
Supportive Environment %
Effective School Leadership %
Strong Family-Community Ties %
Trust %
Average ELA Proficiency
Average Math Proficiency
asian_pacific_2016_num_students
asian_pacific_2017_num_students
avg_math_proficiency_2016
avg_math_proficiency_2016_city_diff
avg_math_proficiency_2016_comp_diff
avg_math_proficiency_2017
avg_math_proficiency_2017_city_diff
avg_math_proficiency_2017_comp_diff
black_2016_avg_ela
black_2016_avg_math
black_2016_incoming_ela
black_2016_incoming_math
black_2016_num_students
black_2017_avg_ela
black_2017_avg_math
black_2017_incoming_ela
black_2017_incoming_math
black_2017_num_students
econ_need_index_2016_city_diff
econ_need_index_2016_dist_diff
econ_need_index_2016_val
econ_need_index_2017_city_diff
econ_need_index_2017_dist_diff
econ_need_index_2017_val
grade_7_2015_enrollment
grade_7_2016_enrollment
grade_7_2017_enrollment
grade_8_2015_enr

/Users/greg.bolla/.local/share/virtualenvs/kaggle-passnyc-hqRn-hsS/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1554: RuntimeWarning: invalid value encountered in double_scalars
  return self.ess/self.df_model
/Users/greg.bolla/.local/share/virtualenvs/kaggle-passnyc-hqRn-hsS/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Supportive Environment Rating_nan
Effective School Leadership Rating_Approaching Target
Effective School Leadership Rating_Exceeding Target
Effective School Leadership Rating_Not Meeting Target
Effective School Leadership Rating_nan
Strong Family-Community Ties Rating_Approaching Target
Strong Family-Community Ties Rating_Exceeding Target
Strong Family-Community Ties Rating_Not Meeting Target
Strong Family-Community Ties Rating_nan
Trust Rating_Approaching Target
Trust Rating_Exceeding Target
Trust Rating_Not Meeting Target
Trust Rating_nan
Student Achievement Rating_Approaching Target
Student Achievement Rating_Exceeding Target
Student Achievement Rating_Not Meeting Target
Student Achievement Rating_nan
dist_to_bronx_hs_of_science
dist_to_brooklyn_latin_school
dist_to_brooklyn_tech_hs
dist_to_hs_for_math_sci_eng
dist_to_hs_of_amer_studies
dist_to_queens_hs_for_sci
dist_to_staten_island_tech
dist_to_stuyvesant_hs
min_dist_to_specialized_school


In [32]:
def get_categorical_model(col_name, pred_train_df, response_train_df):
    model_rows = []
    
    col_list = pred_train_df.columns.values
    
    col_dummy_vars = [var for var in col_list if "{}_".format(col_name) in var]
    x_train_df = pred_train_df[col_dummy_vars]
    x_train_df = sm.add_constant(x_train_df)

    x_model = sm.OLS(response_train_df, x_train_df, missing='drop')
    x_results = x_model.fit()
    
    num_obs = x_results.nobs
    r2 = x_results.rsquared
    adj_r2 = x_results.rsquared_adj
    f_stat = x_results.fvalue
    f_stat_pval = x_results.f_pvalue
    aic = x_results.aic
    bic = x_results.bic
    for i, pred_pval in enumerate(x_results.pvalues):
        if np.isnan(pred_pval):
            print(x_results.summary())
            continue
        model_row = {}
        model_row['model'] = col_name
        if len(x_results.pvalues[x_results.pvalues == pred_pval]) > 1:
            print(x_results.pvalues[x_results.pvalues == pred_pval])
            raise ValueError('multiple preds have same pval')
        pred_col = x_results.pvalues[x_results.pvalues == pred_pval].index[0]
        pred_coef = x_results.params[pred_col]
        
        
        model_row['model_r2'] = r2
        model_row['model_adj_r2'] = adj_r2
        model_row['model_f_stat'] = f_stat
        model_row['model_f_stat_pval'] = f_stat_pval
        model_row['model_aic'] = aic
        model_row['model_bic'] = bic
        model_row['pred_col'] = pred_col
        model_row['pred_coef'] = pred_coef
        model_row['pred_col_pval'] = pred_pval
        if pred_col != 'const':
            model_row['pred_num_miss'] = pred_train_df[pred_col].isnull().sum()
        model_rows.append(model_row)

    return model_rows

In [94]:
cat_pred_df_cols = ["model", "model_r2", "model_adj_r2", "model_f_stat", "model_f_stat_pval", "model_aic", "model_bic",
    "pred_col", "pred_coef", "pred_col_pval", "pred_num_miss"
    ]

cat_pred_df = cat_pred_df[cat_pred_df_cols]

In [95]:
cat_pred_df.to_csv("{}/categorical_single_regression.csv".format(data_dir), index=False)
num_pred_df.to_csv("{}/numerical_single_regression.csv".format(data_dir), index=False)

In [46]:
def get_numerical_model(col_name, pred_train_df, response_train_df):
    model_rows = []
    
    x_train_df = pred_train_df[col_name]
    x_train_df = sm.add_constant(x_train_df)

    x_model = sm.OLS(response_train_df, x_train_df, missing='drop')
    x_results = x_model.fit()
    
    num_obs = x_results.nobs
    r2 = x_results.rsquared
    adj_r2 = x_results.rsquared_adj
    f_stat = x_results.fvalue
    f_stat_pval = x_results.f_pvalue
    aic = x_results.aic
    bic = x_results.bic
    for i, pred_pval in enumerate(x_results.pvalues):
        if np.isnan(pred_pval):
            print(x_results.summary())
            continue
        model_row = {}
        model_row['model'] = col_name
        if len(x_results.pvalues[x_results.pvalues == pred_pval]) > 1:
            print(x_results.pvalues[x_results.pvalues == pred_pval])
            raise ValueError('multiple preds have same pval')
        pred_col = x_results.pvalues[x_results.pvalues == pred_pval].index[0]
        pred_coef = x_results.params[pred_col]
        
        
        model_row['model_r2'] = r2
        model_row['model_adj_r2'] = adj_r2
        model_row['model_f_stat'] = f_stat
        model_row['model_f_stat_pval'] = f_stat_pval
        model_row['model_aic'] = aic
        model_row['model_bic'] = bic
        model_row['pred_col'] = pred_col
        model_row['pred_coef'] = pred_coef
        model_row['pred_col_pval'] = pred_pval
        if pred_col != 'const':
            model_row['pred_num_miss'] = pred_train_df[pred_col].isnull().sum()
        model_rows.append(model_row)

    return model_rows

In [84]:
def get_dist_from_specialized_schools(row):
   
    row_long_lat = (float(row['Latitude']), float(row['Longitude']))

    specialized_school_long_lat = {
        "bronx_hs_of_science" : (40.87833, -73.89083),
        "brooklyn_latin_school" : (40.705, -73.9388889),
        "brooklyn_tech_hs" : (40.6888889, -73.9766667),
        "hs_for_math_sci_eng" : (40.8215, -73.9490),
        "hs_of_amer_studies" : (40.8749, -73.8952),
        "queens_hs_for_sci": (40.699, -73.797),
        "staten_island_tech" : (40.5676, -74.1181),
        "stuyvesant_hs" : (40.7178801, -74.0137509)
    }
    
    #dist_df = pd.DataFrame()
    row = {}
    for specialized_school, specialized_long_lat in specialized_school_long_lat.items():
        row["dist_to_{}".format(specialized_school)] = geopy.distance.vincenty(row_long_lat, specialized_long_lat).miles       
    row["min_dist_to_specialized_school"] = row[min(row, key=row.get)]
    return pd.Series(row)


In [85]:
dist_df = modeling_df.apply(get_dist_from_specialized_schools, axis=1)

In [86]:
dist_df.head()

,dist_to_bronx_hs_of_science,dist_to_brooklyn_latin_school,dist_to_brooklyn_tech_hs,dist_to_hs_for_math_sci_eng,dist_to_hs_of_amer_studies,dist_to_queens_hs_for_sci,dist_to_staten_island_tech,dist_to_stuyvesant_hs,min_dist_to_specialized_school
0,18.275778,6.328220,4.450203,13.670928,17.978016,11.933558,7.300248,6.233516,4.450203
1,21.354160,9.159982,7.650721,16.910698,21.068363,13.207266,6.658060,9.671763,6.658060
2,6.562026,6.708981,7.736581,1.618959,6.236976,11.169889,18.023136,6.297388,1.618959
3,15.051826,3.301437,1.226399,10.471718,14.753907,10.027592,10.065957,3.504831,1.226399
4,11.043313,4.222236,3.942374,6.123651,10.721510,11.193139,13.518514,1.805418,1.805418


In [87]:
example_df = pd.concat([modeling_df, dist_df], axis=1)

In [88]:
example_df.head()

,school_name,dbn,num_testtakers,num_offered,pct_8th_graders_offered,pct_black_hispanic,SED Code,District,Latitude,Longitude,...,Student Achievement Rating_nan,dist_to_bronx_hs_of_science,dist_to_brooklyn_latin_school,dist_to_brooklyn_tech_hs,dist_to_hs_for_math_sci_eng,dist_to_hs_of_amer_studies,dist_to_queens_hs_for_sci,dist_to_staten_island_tech,dist_to_stuyvesant_hs,min_dist_to_specialized_school
0,THE CHRISTA MCAULIFFE SCHOOL\I.S. 187,20K187,251.0,205.0,0.75,0.08,3.320000e+11,20.0,40.627847,-74.004003,...,0.0,18.275778,6.328220,4.450203,13.670928,17.978016,11.933558,7.300248,6.233516,4.450203
1,MARK TWAIN I.S. 239 FOR THE GIFTED & TALENTED,21K239,336.0,196.0,0.46,0.13,3.321000e+11,21.0,40.578660,-73.992391,...,0.0,21.354160,9.159982,7.650721,16.910698,21.068363,13.207266,6.658060,9.671763,6.658060
2,J.H.S. 054 BOOKER T. WASHINGTON,03M054,257.0,150.0,0.53,0.23,3.103000e+11,3.0,40.800512,-73.962802,...,0.0,6.562026,6.708981,7.736581,1.618959,6.236976,11.169889,18.023136,6.297388,1.618959
3,M.S. 51 WILLIAM ALEXANDER,15K051,280.0,122.0,0.33,0.28,3.315000e+11,15.0,40.672180,-73.984625,...,0.0,15.051826,3.301437,1.226399,10.471718,14.753907,10.027592,10.065957,3.504831,1.226399
4,NEW YORK CITY LAB MIDDLE SCHOOL FOR COLLABORAT...,02M312,163.0,113.0,0.62,0.08,3.102000e+11,2.0,40.742571,-74.002371,...,0.0,11.043313,4.222236,3.942374,6.123651,10.721510,11.193139,13.518514,1.805418,1.805418


In [44]:
preds = [
"Average ELA Proficiency", 
"Average Math Proficiency",
"sa_attendance_90plus_2017" , 
"min_dist_to_specialized_school",
"Student Achievement Rating_Approaching Target",
"Student Achievement Rating_Exceeding Target",
"Student Achievement Rating_Not Meeting Target",
"Student Achievement Rating_nan",
"Strong Family-Community Ties Rating_Approaching Target",
"Strong Family-Community Ties Rating_Exceeding Target",
"Strong Family-Community Ties Rating_Not Meeting Target",
"Strong Family-Community Ties Rating_nan",
"pct_8th_graders_w_hs_credit_2017",
"borough_bronx",
"borough_manhattan",
"borough_queens",
"borough_staten_island"
]

cols_to_standardize = [
    "Average ELA Proficiency", 
    "Average Math Proficiency",
    "sa_attendance_90plus_2017",
    "min_dist_to_specialized_school",
    "pct_8th_graders_w_hs_credit_2017"
]
#x_train = pred_train[preds]

#len(x_train.dropna())
#mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(x_train, response_train)
 

#sm_x_train = sm.add_constant(sm_x_train)

In [109]:
subsetted_modeling_df = modeling_df[]

subsetted_modeling_df = subsetted_modeling_df.dropna()
subsetted_pred_df = subsetted_modeling_df[preds]
subsetted_response_df = subsetted_modeling_df["perc_testtakers_quartile"]
subsetted_pred_train, subsetted_pred_test, subsetted_response_train, subsetted_response_test = train_test_split(subsetted_pred_df, subsetted_response_df, test_size=0.2, random_state=223)


#multinom_response_df = modeling_df["perc_testtakers_quartile"]

In [110]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(subsetted_pred_train, subsetted_response_train)



In [115]:
print("Logistic regression Train Accuracy :: ", metrics.accuracy_score(subsetted_response_train, mul_lr.predict(subsetted_pred_train)))
print("Logistic regression Test Accuracy :: ", metrics.accuracy_score(subsetted_response_test, mul_lr.predict(subsetted_pred_test)))

 

Logistic regression Train Accuracy ::  0.4964871194379391
Logistic regression Test Accuracy ::  0.48598130841121495


In [121]:
print(mul_lr.predict(subsetted_pred_train))

[4. 4. 1. 2. 1. 2. 1. 1. 4. 1. 4. 1. 4. 1. 2. 1. 2. 4. 1. 1. 4. 4. 1. 1.
 1. 3. 2. 1. 4. 2. 3. 4. 3. 1. 4. 1. 4. 3. 1. 2. 1. 1. 3. 1. 3. 1. 1. 4.
 3. 4. 1. 1. 2. 1. 4. 2. 1. 4. 3. 3. 3. 2. 3. 1. 1. 4. 1. 3. 4. 3. 3. 1.
 3. 1. 1. 2. 4. 4. 4. 3. 4. 1. 4. 4. 1. 4. 3. 4. 1. 4. 4. 2. 2. 1. 4. 4.
 2. 3. 4. 3. 1. 1. 2. 2. 4. 4. 3. 1. 1. 1. 1. 3. 2. 2. 2. 2. 2. 2. 3. 2.
 2. 4. 2. 1. 2. 4. 2. 4. 2. 1. 1. 1. 1. 4. 4. 3. 1. 4. 1. 1. 4. 2. 4. 1.
 4. 4. 3. 1. 4. 1. 2. 2. 2. 4. 3. 1. 4. 4. 2. 3. 1. 3. 4. 1. 3. 1. 1. 4.
 4. 2. 2. 2. 4. 1. 4. 1. 4. 1. 1. 2. 2. 2. 2. 4. 2. 3. 4. 2. 3. 1. 3. 4.
 1. 4. 1. 3. 2. 1. 3. 3. 2. 1. 4. 1. 4. 1. 4. 1. 4. 4. 2. 1. 3. 3. 4. 2.
 4. 1. 4. 2. 1. 2. 1. 3. 2. 1. 1. 4. 1. 2. 1. 3. 2. 3. 3. 1. 4. 2. 4. 4.
 2. 1. 1. 4. 3. 4. 2. 1. 1. 4. 3. 1. 2. 2. 4. 4. 3. 1. 1. 2. 2. 4. 2. 1.
 4. 1. 4. 1. 4. 1. 1. 4. 2. 3. 1. 1. 4. 1. 2. 3. 4. 3. 4. 4. 1. 2. 1. 4.
 4. 2. 4. 2. 2. 3. 4. 1. 2. 1. 4. 2. 1. 4. 2. 2. 1. 3. 4. 2. 2. 4. 2. 2.
 2. 1. 1. 1. 2. 3. 1. 1. 4. 4. 1. 1. 3. 2. 4. 1. 2.

In [120]:
print(subsetted_response_train)

6      4.0
153    1.0
345    2.0
160    1.0
166    1.0
451    3.0
156    1.0
349    4.0
322    4.0
326    2.0
39     3.0
203    1.0
64     4.0
492    4.0
56     3.0
317    2.0
301    2.0
342    2.0
475    3.0
249    1.0
66     3.0
285    3.0
292    1.0
455    2.0
254    1.0
60     3.0
294    2.0
307    1.0
162    3.0
35     4.0
      ... 
27     4.0
199    2.0
515    4.0
351    3.0
104    3.0
187    1.0
149    1.0
450    3.0
338    2.0
315    1.0
434    4.0
172    1.0
214    1.0
232    2.0
481    3.0
403    2.0
116    1.0
272    1.0
98     4.0
14     4.0
408    4.0
76     4.0
206    3.0
284    4.0
159    1.0
454    2.0
177    1.0
112    3.0
17     4.0
483    1.0
Name: perc_testtakers_quartile, Length: 427, dtype: float64


In [123]:
modeling_df.City.value_counts()

BROOKLYN               185
BRONX                  137
NEW YORK               115
STATEN ISLAND           17
JAMAICA                 14
FLUSHING                13
LONG ISLAND CITY        10
FAR ROCKAWAY             6
SPRINGFIELD GARDENS      5
ROCKAWAY PARK            4
ELMHURST                 4
MIDDLE VILLAGE           4
ASTORIA                  4
JACKSON HEIGHTS          3
BAYSIDE                  3
OZONE PARK               3
HOWARD BEACH             3
QUEENS VILLAGE           2
ROSEDALE                 2
GLENDALE                 2
RIDGEWOOD                2
REGO PARK                2
SOUTH OZONE PARK         2
EAST ELMHURST            2
ARVERNE                  2
SAINT ALBANS             2
BELLEROSE                2
FOREST HILLS             1
ROCKAWAY BEACH           1
MASPETH                  1
ROOSEVELT ISLAND         1
CORONA                   1
CAMBRIA HEIGHTS          1
FLORAL PARK              1
HOLLIS                   1
WHITESTONE               1
LITTLE NECK              1
B

In [37]:
col = "Average ELA Proficiency"
pred_train[col].isnull().sum()

0